## Вопросы для самопроверки:
* что такое обучени с подкреплением (reinforcement learning)?
* что такое среда?
* что такое агент?
* что такое награда, какая она может быть?

In [1]:
import gym

#create a single game instance
env = gym.make("Taxi-v2")

#start new game
env.reset();

[2017-12-04 23:19:01,777] Making new env: Taxi-v2


In [2]:
print("initial observation code:", env.reset())
print('printing observation:')
env.render()
print("observations:", env.observation_space, 'n=', env.observation_space.n)
print("actions:", env.action_space, 'n=', env.action_space.n)

initial observation code: 211
printing observation:
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

observations: Discrete(500) n= 500
actions: Discrete(6) n= 6


In [3]:
import numpy as np
n_states = env.observation_space.n
n_actions = env.action_space.n
def get_random_policy():
    """
    Build a numpy array representing agent policy.
    This array must have one element per each of 16 environment states.
    Element must be an integer from 0 to 3, representing action
    to take from that state.
    """
    return np.random.choice(range(n_actions), size=n_states)

In [4]:
np.random.seed(1234)
policies = [get_random_policy() for i in range(10**4)]
assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should match n_actions-1'
action_probas = np.unique(policies, return_counts=True)[-1] /10**4. /n_states
print("Action frequencies over 10^4 samples:",action_probas)
assert np.allclose(action_probas, [1. / n_actions] * n_actions, atol=0.05), "The policies aren't uniformly random (maybe it's just an extremely bad luck)"
print("Seems fine!")

Action frequencies over 10^4 samples: [ 0.1668264  0.1667818  0.166578   0.166587   0.166508   0.1667188]
Seems fine!


### Let's evaluate!
* Implement a simple function that runs one game and returns the total reward

In [6]:
def sample_reward(env, policy, max_steps=100):
    """
    Interact with an environment, return sum of all rewards.
    If game doesn't end on t_max (e.g. agent walks into a wall), 
    force end the game and return whatever reward you got so far.
    Tip: see signature of env.step(...) method above.
    """
    
    n_steps = 0
    
    s = env.reset()
    state = 0
    total_reward = 0
    done = False
    
    while(not done and n_steps <= max_steps):
        state, reward, done, _ = env.step(policy[n_steps])
        total_reward += reward
        n_steps += 1
    
    return total_reward

In [7]:
print("generating 10^3 sessions...")
rewards = [sample_reward(env, get_random_policy()) for _ in range(10**3)]
assert all([type(r) in (int, float) for r in rewards]), 'sample_reward must return a single number'
print("Looks good!")

generating 10^3 sessions...
Looks good!


In [9]:
def evaluate(policy, n_times=100):
    """Run several evaluations and average the score the policy gets."""
    rewards = [sample_reward(env, policy) for i in range(n_times)]
    return float(np.mean(rewards))       

### Main loop

In [11]:
best_policy = None
best_score = -float('inf')

from tqdm import tqdm
for i in tqdm(range(10000)):
    policy = get_random_policy()
    score = evaluate(policy)
    if score > best_score:
        best_score = score
        best_policy = policy
        print("New best score:", score)

  0%|          | 2/10000 [00:00<20:34,  8.10it/s]

New best score: -399.53
New best score: -393.88


  0%|          | 6/10000 [00:00<19:47,  8.42it/s]

New best score: -296.93


  1%|          | 84/10000 [00:09<19:08,  8.63it/s]

...

 39%|███▊      | 3869/10000 [07:19<11:36,  8.80it/s]

New best score: -252.48


 54%|█████▍    | 5406/10000 [10:18<08:45,  8.74it/s]

New best score: -250.28


100%|██████████| 10000/10000 [19:11<00:00,  8.68it/s]


In [12]:
print(best_score)

-250.28


# Part II Genetic algorithm 

The next task is to devise some more effecient way to perform policy search.
We'll do that with a bare-bones evolutionary algorithm.
[unless you're feeling masochistic and wish to do something entirely different which is bonus points if it works]

In [13]:
def crossover(policy1, policy2, p=0.5):
    """
    for each state, with probability p take action from policy1, else policy2
    """
    q = 1 - p
    
    cross_policy = np.zeros(np.shape(policy1))
    policy_filter = np.random.choice([False, True], size=n_states, p=[q, p])
    inverted_policy_filter = policy_filter == 0
    cross_policy[policy_filter] = policy1[policy_filter]
    cross_policy[inverted_policy_filter] = policy2[inverted_policy_filter]
    return cross_policy

In [14]:
def mutation(policy, p=0.1):
    """
    for each state, with probability p replace action with random action
    Tip: mutation can be written as crossover with random policy
    """
    mutated_policy = crossover(get_random_policy(), policy, p)
    return mutated_policy
    

In [15]:
np.random.seed(1234)
policies = [crossover(get_random_policy(), get_random_policy()) 
            for i in range(10**4)]

assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should be n_actions-1'

assert any([np.mean(crossover(np.zeros(n_states), np.ones(n_states))) not in (0, 1)
               for _ in range(100)]), "Make sure your crossover changes each action independently"
print("Seems fine!")

Seems fine!


In [16]:

n_epochs = 100 #how many cycles to make
pool_size = 100 #how many policies to maintain
n_crossovers = 50 #how many crossovers to make on each step
n_mutations = 50 #how many mutations to make on each tick


In [17]:
print("initializing...")
pool = [get_random_policy() for i in range(pool_size)]
pool_scores = [evaluate(i) for i in pool]

initializing...


In [18]:
assert type(pool) == type(pool_scores) == list
assert len(pool) == len(pool_scores) == pool_size
assert all([type(score) in (float, int) for score in pool_scores])

In [19]:
#main loop
best_score_1 = -float('inf')
for epoch in tqdm(range(n_epochs)):
    
    
    
    
    crossovered = [crossover(pool[i], pool[-i - 1]) for i in np.random.choice(range(pool_size), n_crossovers)]
    mutated = [mutation(pool[i]) for i in np.random.choice(range(pool_size), n_mutations)]
    
    assert type(crossovered) == type(mutated) == list
    
    #add new policies to the pool
    pool = pool + crossovered + mutated
    pool_scores = [evaluate(i) for i in pool]
    
    #select pool_size best policies
    selected_indices = np.argsort(pool_scores)[-pool_size:]
    pool = [pool[i] for i in selected_indices]
    pool_scores = [pool_scores[i] for i in selected_indices]

    #print the best policy so far (last in ascending score order)
    if pool_scores[-1] > best_score_1:
        print("Epoch %s:"%epoch)
        print("best score:", pool_scores[-1])

  1%|          | 1/100 [00:19<32:48, 19.88s/it]

Epoch 0:
best score: -298.19


  2%|▏         | 2/100 [00:39<32:08, 19.68s/it]

Epoch 1:
best score: -296.57


  3%|▎         | 3/100 [00:59<31:54, 19.73s/it]

...

 54%|█████▍    | 54/100 [17:53<15:14, 19.88s/it]

Epoch 53:
best score: -101.0


 55%|█████▌    | 55/100 [18:13<14:54, 19.89s/it]

Epoch 54:
best score: -101.0


KeyboardInterrupt: 

Ссылка на фидбек по семинару: [link](https://docs.google.com/forms/d/e/1FAIpQLSf-08wFrEke6zKlysETYiqAjH5CRXtOKut5Q77Tr5rdVId7zA/)